In [ ]:
import xarray as xr
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os,errno
import sys
import numpy as np

dir_data='/Users/ahenny/'

In [ ]:
import cdsapi

#yrs=np.arange(1952,2023,1)
yrs=np.arange(2023,2024,1)
for l in range(len(yrs)):
    year=yrs[l]
    
    c = cdsapi.Client()

    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'total_precipitation',
            'year': [
                str(year),
            ],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'format': 'netcdf',
        },
        'era5.1hr.precip.nc')


    ds=xr.open_dataset('/Users/ahenny/paper_1/era5.1hr.precip.nc')

    days_year=365
    count=0
    dates_list=[]
    if year%4==0:
        days_year=366
    for i in range(days_year):
        print(i)
        date_start=dt.datetime(year,1,1)+dt.timedelta(days=i)
        date_range=[date_start+dt.timedelta(hours=x) for x in range(24)]
        overlap=[x for x in ds.time.values if pd.to_datetime(x) in date_range]
        if len(overlap)<24:
            print('FAIL')
        else:
            precip=ds['tp'].sel(time=date_range).sum(dim='time')*1000.#convert to mm
            lons_east=[x for x in precip.longitude.values if x>=180.]
            lons_west=[x for x in precip.longitude.values if x<180.]
            p_east=precip.sel(longitude=lons_east)
            p_west=precip.sel(longitude=lons_west)
            p_east['longitude']=[x-360. for x in lons_east]
            precip=xr.concat([p_east,p_west],dim='longitude')

            if count==0:
                precip_concat=precip
                count=1
            else:
                precip_concat=xr.concat([precip_concat,precip],dim='time')

            dates_list.append(date_start)
    precip_concat['time']=dates_list

    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time
    dk.coords['lat']=precip_concat.latitude.values
    dk.coords['lon']=precip_concat.longitude.values
    dk['precip'].attrs["units"]='mm*day-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    try:
        os.remove(dir_data+'era_5_precip_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'era_5_precip_'+str(year)+'.nc',mode='w',format='NETCDF4')

    os.remove('/Users/ahenny/paper_1/era5.1hr.precip.nc')

In [ ]:
os.remove('/Users/ahenny/paper_1/era5.1hr.precip.nc')